# Import Required Libraries
Import libraries such as pandas, matplotlib, and numpy for data analysis and visualization.

In [ ]:
# Import Required Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

# Parse Log File and Extract Data
Extract relevant data directly from the .log file for analysis.

In [ ]:
# Define the log file path
log_file_path = '/home/imes-humanoid2/humanoid/avp_teleoperate/teleop/logs/pose_analysis_20250522_110551.log'

# Initialize lists to store extracted data
timestamps = []
left_wrist_positions = []
right_wrist_positions = []

# Define regex patterns to extract data
timestamp_pattern = re.compile(r'^(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3})')
left_wrist_pattern = re.compile(r'Left Wrist Analysis:.*?Position: \([\d\.-]+), ([\d\.-]+), ([\d\.-]+)\')
right_wrist_pattern = re.compile(r'Right Wrist Analysis:.*?Position: \([\d\.-]+), ([\d\.-]+), ([\d\.-]+)\')

# Read the log file
with open(log_file_path, 'r') as file:
    for line in file:
        # Extract timestamp
        timestamp_match = timestamp_pattern.match(line)
        if timestamp_match:
            timestamps.append(timestamp_match.group(1))

        # Extract left wrist position
        left_wrist_match = left_wrist_pattern.search(line)
        if left_wrist_match:
            left_wrist_positions.append([float(left_wrist_match.group(1)), float(left_wrist_match.group(2)), float(left_wrist_match.group(3))])

        # Extract right wrist position
        right_wrist_match = right_wrist_pattern.search(line)
        if right_wrist_match:
            right_wrist_positions.append([float(right_wrist_match.group(1)), float(right_wrist_match.group(2)), float(right_wrist_match.group(3))])

# Create a DataFrame
data = pd.DataFrame({
    'timestamp': timestamps[:len(left_wrist_positions)],
    'left_wrist_x': [pos[0] for pos in left_wrist_positions],
    'left_wrist_y': [pos[1] for pos in left_wrist_positions],
    'left_wrist_z': [pos[2] for pos in left_wrist_positions],
    'right_wrist_x': [pos[0] for pos in right_wrist_positions],
    'right_wrist_y': [pos[1] for pos in right_wrist_positions],
    'right_wrist_z': [pos[2] for pos in right_wrist_positions]
})

# Display the first few rows of the data
print(data.head())

# Parse Hand Tracking Data
Extract relevant hand tracking data from the log files, such as hand positions, orientations, and timestamps.

In [ ]:
# Parse Hand Tracking Data
# Assuming the log file contains columns like 'timestamp', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z'
hand_tracking_data = data[['timestamp', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z']]

# Convert timestamp to a datetime object for easier analysis
hand_tracking_data['timestamp'] = pd.to_datetime(hand_tracking_data['timestamp'])

# Display the parsed hand tracking data
print(hand_tracking_data.head())

# Visualize Hand Tracking Data
Create plots to visualize the hand tracking data over time, including position and orientation changes.

In [ ]:
# Visualize Hand Tracking Data
plt.figure(figsize=(10, 6))

# Plot hand positions over time
plt.plot(hand_tracking_data['timestamp'], hand_tracking_data['left_wrist_x'], label='X Position')
plt.plot(hand_tracking_data['timestamp'], hand_tracking_data['left_wrist_y'], label='Y Position')
plt.plot(hand_tracking_data['timestamp'], hand_tracking_data['left_wrist_z'], label='Z Position')

plt.title('Hand Tracking Data Over Time')
plt.xlabel('Timestamp')
plt.ylabel('Position')
plt.legend()
plt.grid()
plt.show()

# Analyze Discrepancies in Hand Tracking
Identify anomalies or inconsistencies in the hand tracking data, such as sudden jumps or unrealistic values.

In [ ]:
# Analyze Discrepancies in Hand Tracking
# Calculate differences between consecutive positions to identify sudden jumps
hand_tracking_data['position_diff'] = hand_tracking_data[['left_wrist_x', 'left_wrist_y', 'left_wrist_z']].diff().abs().sum(axis=1)

# Identify rows where the position difference exceeds a threshold (e.g., 10 units)
anomalies = hand_tracking_data[hand_tracking_data['position_diff'] > 10]

# Display anomalies
print("Anomalies in hand tracking data:")
print(anomalies)

# Correlate Hand Tracking with Robot Movements
Compare the hand tracking data with the robot's hand movements to identify patterns or mismatches.

In [ ]:
# Correlate Hand Tracking with Robot Movements
# Assuming the log file also contains robot movement data in columns like 'robot_hand_position_x', 'robot_hand_position_y', 'robot_hand_position_z'
robot_movement_data = data[['timestamp', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z']]

# Merge hand tracking data with robot movement data on the timestamp
merged_data = pd.merge(hand_tracking_data, robot_movement_data, on='timestamp')

# Calculate differences between hand tracking and robot movements
merged_data['tracking_robot_diff'] = merged_data[['left_wrist_x', 'right_wrist_x']].diff().abs().sum(axis=1)

# Display the merged data with differences
print(merged_data.head())

# Identify Potential Causes of Errors
Analyze the visualizations and correlations to hypothesize whether the strange behavior is caused by bad hand tracking or other factors.

In [ ]:
# Identify Potential Causes of Errors
# Hypothesize based on visualizations and discrepancies
# For example, if anomalies in hand tracking data align with robot movement errors, it may indicate bad hand tracking
if not anomalies.empty:
    print("Potential causes of errors identified in hand tracking data.")
else:
    print("No significant anomalies detected in hand tracking data.")

# Parse Log File and Save as CSV
Extract relevant data from the log file and save it as a CSV file for further analysis.

In [ ]:
# Identify Potential Causes
low_confidence_threshold = 0.5
sudden_jump_threshold = 0.1  # Example threshold for sudden position changes

# Identify low confidence periods
low_confidence_periods = df[df["confidence"] < low_confidence_threshold]

# Identify sudden jumps in position
df["position_change"] = df["actual_position"].diff().apply(lambda x: np.linalg.norm(x) if isinstance(x, tuple) else 0)
sudden_jumps = df[df["position_change"] > sudden_jump_threshold]

print("Low Confidence Periods:")
print(low_confidence_periods)

print("\nSudden Jumps in Position:")
print(sudden_jumps)